In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42)
# sub 훈련세트, val 검증세트로 나눔
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)
print(sub_input.shape, val_input.shape)
# 훈련세트와 검증 세트로 모델 훈련, 평가
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

(4157, 3) (1040, 3)
0.9971133028626413
0.864423076923077


In [ ]:
# 교차 검증

from sklearn.model_selection import cross_validate
scores=cross_validate(dt,train_input,train_target)
print(scores)
# 교차 검증의 최종 점수
import numpy as np
print(np.mean(scores['test_score']))
# 교차 검증할때 훈련 세트를 섞으려면 분할기를 지정해야함.
# 회귀모델은 KFold , 분류모델은 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))
# 훈련 세트를 섞은 후 10폴드교차검증 수
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

{'fit_time': array([0.01063704, 0.00987005, 0.01035547, 0.01040435, 0.00981212]), 'score_time': array([0.00247431, 0.00231552, 0.00243521, 0.00232244, 0.00234699]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
0.855300214703487


In [ ]:
# 하이퍼파라미터 튜닝

from sklearn.model_selection import GridSearchCV
# 탐색할 매개변수 : 탐색할 값
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
# params와 모델을 그리드 서치 객체에 전달
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
# 교차 검증 점수가 가장 높은 모델의 매개변수 조합으로 다시 훈련한 모델이
# best_estimator_에 저장되어 있음.
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
# 최적의 매개변수가 저장된 곳
print(gs.best_params_)
# 각 매개변수에서 수행한 교차 검증의 평균 점수가 저장된 곳.
print(gs.cv_results_['mean_test_score'])
# 가장 높은 값의 인덱스를 골라 매개변수 출력
print(gs.cv_results_['params'][gs.best_index_])

# 매개변수 조합 탐색
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
# 최상의 매개변수 조합
print(gs.best_params_)
# 최상의 교차 검증 점수
print(np.max(gs.cv_results_['mean_test_score']))

0.9615162593804117
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}
{'max_depth': 14, 'min_impurity_decrease': np.float64(0.0004), 'min_samples_split': 12}
0.8683865773302731


In [ ]:
# 랜덤 서치

from scipy.stats import uniform, randint
# 정수 샘플링
rgen = randint(0, 10)
rgen.rvs(10)
# 1000개 샘플링하여 각 숫자의 개수 세기
np.unique(rgen.rvs(1000),return_counts=True)
# 실수 샘플
ugen=uniform(0,1)
ugen.rvs(10)

# 랜덤 서치 알고리즘 사용해 최적의 모델 찾기.
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)
# 최적의 매개변수 조합
print(gs.best_params_)
# 최고의 교차검증 점수
print(np.max(gs.cv_results_['mean_test_score']))
# 저장된 최적의 모델 성능 확인
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86


In [ ]:
# 확인 문제
rs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random',random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
rs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

{'max_depth': 39, 'min_impurity_decrease': np.float64(0.00034102546602601173), 'min_samples_leaf': 7, 'min_samples_split': 13}
0.8695428296438884
0.86
